In [62]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import scipy as sc
from scipy import stats
import pandas as pd
import pickle
import tqdm

from typing import List, Callable

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

#from rulefit import RuleFit
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import BayesianRidge, LinearRegression, LogisticRegression, ElasticNet
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, TunedThresholdClassifierCV
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
#from category_encoders import JamesSteinEncoder, CatBoostEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

import matplotlib.pyplot as plt

# add ../src to python path
sys.path.insert(0, os.path.join(os.path.abspath('.'),'..', 'src'))

import tree_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import seaborn as sns

In [64]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import brier_score_loss
from sklearn.calibration import calibration_curve

In [65]:
# Try from sklego.meta import HierarchicalClassifier
# hc = HierarchicalClassifier(
#    estimator=LogisticRegression(),
#    groups=groups
#).fit(X, y)
#hc.estimators_

In [66]:
data_path =  r"J:\Onderzoek\21-763_rvanes_MiniECG-2-Data\E_ResearchData\2_ResearchData\Analysis"
file_name =  r"input_decision_tree_complete.pkl"

num_splits = 10
num_repeats = 10
MAX_FEATURES = 100 # only relevant if USE_LEAD_COMBOS is True
USE_CLASS_WEIGHT = False
ALL_FEATURES = True
USE_REDUCED_LABELS = True
USE_CALIBRATED_CLASSIFIER = True
USE_LEAD_COMBOS = False
COMBO_WITH_FEATURE_FILTER = False 
ALL_FEATURES_STRING = "_useAllvars" if ALL_FEATURES else ""
CLASS_WEIGHT_STRING = "_withClassWeights" if USE_CLASS_WEIGHT else ""
REDUCED_LABEL_STRING = "_withReducedLabels" if USE_REDUCED_LABELS else ""
CALIBRATED_CLASSIFIER_STRING = "_withCalibratedClassifier" if USE_CALIBRATED_CLASSIFIER else ""
FEATURE_COMBO_STRING = "_withLeadCombos" if USE_LEAD_COMBOS else ""

In [67]:
with open(os.path.join(data_path, file_name), 'rb') as f:
    input_decision_tree_complete = pickle.load(f)

In [68]:
DATA = pd.DataFrame(input_decision_tree_complete).T

In [69]:
morphology_columns = [c for c in DATA.columns if 'morphology' in c]
lead_columns = [c for c in DATA.columns if ('lead' in c) & ('morphology' not in c)]

for c in morphology_columns:
    DATA.loc[:, c] = DATA[c].apply(lambda x: x[0].strip(",").strip(" "))
    DATA.loc[:, c] = DATA[c].apply(lambda x: x if x.strip()!="" else "none")

In [70]:
morphology_values = []
for c in morphology_columns:
    morphology_values.extend(DATA[c].unique().tolist())
morphology_values = list(set(morphology_values))

In [71]:
ComboFunctions = {
    'AbsFactor' : lambda x,y: abs(x-y) # np.sign(x*y)*np.sqrt(np.abs(x*y))
}

In [72]:
if USE_LEAD_COMBOS:
    Enriched_DF, new_columns = tree_utils.create_feature_combinations(df=DATA[lead_columns], 
                                           lambda_functions=ComboFunctions)
    DATA = DATA.drop(lead_columns, axis=1)
    DATA = pd.concat([DATA, Enriched_DF], axis=1)


# Model prepping

In [73]:
# TODO: need to add a feature combiner, perhaps use PySR or GpLearn, focus on vectors

impute_kwargs = {
    'estimator': LinearRegression(), 
    'random_state':7,
    'imputation_order': 'ascending', 
    'skip_complete': True,
    'max_iter': 2000,
    'initial_strategy': 'median',
    'add_indicator': True
}
gradientboosting_kwargs = {
    'n_estimators': 500, 
    'max_depth': 10, 
    'learning_rate':0.01,
    'max_leaf_nodes':40,
    'random_state': 7
}
randomforest_kwargs = {
    'n_estimators': 150,
    'max_depth': 6,
    'min_samples_split': 10, 
    'min_samples_leaf': 10,
    'random_state': 7
}
rulefit_kwargs={
    'tree_size': 10,
    'max_rules': 100,
    'tree_generator': GradientBoostingClassifier(**gradientboosting_kwargs)
}
decisiontree_kwargs = {
    'criterion':'gini', 
    'splitter':'best', 
    'max_depth':10, 
    'min_samples_split':10, 
    'min_samples_leaf': 5, 
    'min_weight_fraction_leaf':0.0, 
    'max_features':None, 
    'random_state':7, 
    'max_leaf_nodes':50,
    'class_weight': 'balanced'
}
xgboost_kwargs = {
    'n_estimators': 150,
    'max_depth': 6,
    'max_leaves': 50,
    'learning_rate': 2e-3,
    'gamma': 0.4,
    'subsample': 0.55,
    'colsample_bytree':0.85,
    'reg_alpha': 0.005
}
logistic_kwargs = {
    'penalty': 'elasticnet', 
    'dual': False, 
    'tol': 0.0001, 
    'C':1.0, 
    'fit_intercept': True, 
    'intercept_scaling':1, 
    'class_weight':None, 
    'random_state':7, 
    'solver': 'saga', 
    'max_iter':2000, 
    'verbose': 0, 
    'warm_start': False, 
    'n_jobs':-1, 
    'l1_ratio':0.5
}
calibration_kwargs ={
    'method': 'sigmoid',
    'n_jobs': -1, 
    'ensemble': True,
    'cv': 5
}


In [74]:
OrdEncoder = OrdinalEncoder(
    categories='auto',
    dtype=int,
    handle_unknown='use_encoded_value',
    unknown_value=-2,
    encoded_missing_value=-1,
)
PipeOrdEncoder = ColumnTransformer([("cat_encoder", OrdEncoder, morphology_columns)],
                                   remainder='passthrough')

FFilter = SelectFromModel(estimator=LogisticRegression(**logistic_kwargs),
                            max_features=MAX_FEATURES)

_imputer = IterativeImputer(**impute_kwargs)

if ALL_FEATURES:
    _cat_enc = PipeOrdEncoder
else:
    _cat_enc = None

if (USE_LEAD_COMBOS) & (COMBO_WITH_FEATURE_FILTER):
    _feature_filter = FFilter
    _imputer = SimpleImputer(strategy='median', add_indicator=True)
else:
    _feature_filter = None

if USE_CALIBRATED_CLASSIFIER:
    RF_clf = CalibratedClassifierCV(RandomForestClassifier(**randomforest_kwargs), 
                                    **calibration_kwargs)
    GBC_clf = CalibratedClassifierCV(GradientBoostingClassifier(**gradientboosting_kwargs),
                                     **calibration_kwargs)
    XGB_clf = CalibratedClassifierCV(XGBClassifier(**xgboost_kwargs),
                                     **calibration_kwargs)
    DT_clf = CalibratedClassifierCV(DecisionTreeClassifier(**decisiontree_kwargs),
                                     **calibration_kwargs)
    LR_clf = CalibratedClassifierCV(LogisticRegression(**logistic_kwargs),
                                     **calibration_kwargs)
else:
    RF_clf = RandomForestClassifier(**randomforest_kwargs)
    GBC_clf = GradientBoostingClassifier(**gradientboosting_kwargs)
    XGB_clf = tree_utils.GenericCalibratedClassifier(XGBClassifier(**xgboost_kwargs))
    DT_clf = DecisionTreeClassifier(**decisiontree_kwargs)
    LR_clf = LogisticRegression(**logistic_kwargs)

le_pipe_rf = Pipeline([
    ("CatEncoder", _cat_enc),
    ("Impute", _imputer),
    ("FeatureFilter", _feature_filter),
    ("RandomForest", RF_clf)])

le_pipe_gbc = Pipeline([
    ("CatEncoder", _cat_enc),
    ("Impute", _imputer),
    ("FeatureFilter", _feature_filter),
    ("GradientBoosting", GBC_clf)])

le_pipe_xgb = Pipeline([
    ("CatEncoder", _cat_enc),
    ("Impute", _imputer),
    ("FeatureFilter", _feature_filter),
    ("XGBoost", XGB_clf)])

le_pipe_dt = Pipeline([
    ("CatEncoder", _cat_enc),
    ("Impute", _imputer),
    ("FeatureFilter", _feature_filter),
    ("DecisionTree", DT_clf)])

le_pipe_lr = Pipeline([
    ("CatEncoder", _cat_enc),
    ("Impute", _imputer),
    ("FeatureFilter", _feature_filter),
    ("LogisticRegression", LR_clf)])

PipeDict = {}
#PipeDict['rf'] = le_pipe_rf
#PipeDict['gbc'] = le_pipe_gbc
PipeDict['xgb'] = le_pipe_xgb
PipeDict['dt'] = le_pipe_dt
PipeDict['lr'] = le_pipe_lr


# Axis model

In [75]:
target_col = "Heart Axis Diagnosis"
if not USE_REDUCED_LABELS:
    target_inclusion = ['Left', 'Normal', 'Right']
else:
    target_inclusion = ['Left', 'Normal', 'Right', 'Extreme']
    
Reduction_map = {'Left': 'Disease', 
                 'Right': 'Disease',
                 'Extreme': 'Disease',
                 'Normal': 'Normal'}
if ALL_FEATURES:
    features_to_use = []
else:
    features_to_use = ['qrs_vector mean lead_0',
                     'p_vector mean lead_0',
                     't_vector mean lead_0',
                     'qrs_vector mean lead_1',
                     'p_vector mean lead_1',
                     't_vector mean lead_1',
                     'qrs_vector mean lead_2',
                     'p_vector mean lead_2',
                     't_vector mean lead_2',
                     'qrs_vector mean lead_3',
                     'p_vector mean lead_3',
                     't_vector mean lead_3',
                     'qrs_vector mean lead_4',
                     'p_vector mean lead_4',
                     't_vector mean lead_4',
                     'qrs_vector mean lead_5',
                     'p_vector mean lead_5',
                     't_vector mean lead_5',
                     'qrs_vector mean lead_6',
                     'p_vector mean lead_6',
                     't_vector mean lead_6',
                     'qrs_vector mean lead_7',
                     'p_vector mean lead_7',
                     't_vector mean lead_7'
                       ]

In [76]:
if len(features_to_use)==0:
    meas_cols = [c for c in DATA.columns if ('Dataset' not in c) 
                 & (target_col not in c)
                 & ('Diagnosis' not in c)]
else:
    meas_cols = features_to_use
    
fstring = f"AXIS_{CLASS_WEIGHT_STRING}{ALL_FEATURES_STRING}{REDUCED_LABEL_STRING}{CALIBRATED_CLASSIFIER_STRING}{FEATURE_COMBO_STRING}"
os.makedirs(os.path.join(data_path, fstring), exist_ok=True)

AXIS_DATA = DATA.loc[DATA[target_col].isin(target_inclusion), meas_cols+[target_col]+['Dataset']]
if USE_REDUCED_LABELS:
    AXIS_DATA.loc[:, target_col] = AXIS_DATA[target_col].map(Reduction_map)
    
AXIS_DATA.to_parquet(os.path.join(data_path, fstring, 'DATA.parquet'))
AXIS_DATA = AXIS_DATA.drop('Dataset', axis=1)

### TRAINING LOOP

In [77]:
splitter = RepeatedStratifiedKFold(n_splits=num_splits, n_repeats=num_repeats, random_state=7)

X = AXIS_DATA.iloc[:, :-1]
Y = AXIS_DATA.iloc[:,-1]

lb = LabelBinarizer()
lbe = LabelEncoder()

 #(RES_LIST_AXIS[0]['Y_test'])
Yenc = lbe.fit_transform(Y.values) #(RES_LIST_AXIS[0]['Y_test'])
y_bin = lb.fit(Yenc)
ClassMap_AXIS = {i:c for i,c in enumerate(lbe.classes_)}

In [78]:
ClassMap_AXIS

{0: 'Disease', 1: 'Normal'}

In [79]:
RES_LIST_AXIS, RES_AXIS_DF, RES_AXIS_INDCS_DF = tree_utils.training_loop(X, Yenc, splitter, 
                              PipeDict, 
                              use_class_weights=USE_CLASS_WEIGHT, ClassMap=ClassMap_AXIS,
                              num_splits=num_splits, num_repeats=num_repeats, make_df=True)

100%|██████████| 100/100 [34:02<00:00, 20.43s/it]


In [80]:
RES_AXIS_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS.parquet"))
RES_AXIS_INDCS_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS_w_indices.parquet"))

### make roc and precision recall curves

In [81]:
Y.value_counts()

Heart Axis Diagnosis
Normal     1245
Disease     472
Name: count, dtype: int64

In [82]:
n_classes = len(lb.classes_)
colors = ['blue', 'green', 'red', 'yellow', 'magenta', 'cyan', 'black']

In [83]:
PLOTS_AXIS = tree_utils.make_plots(RES_LIST_AXIS, lb,  n_classes, 
                                   colors, ClassMap_AXIS,
                                   output_map=os.path.join(data_path, fstring),
                                   show_plot=False, 
                                   models = list(PipeDict.keys()),
                                   plot_title="Heart Axis")
perf_list = tree_utils.get_performance(RES_LIST_AXIS, 
                                       threshold=1/n_classes,
                                       ClassMap=ClassMap_AXIS, 
                                       models = list(PipeDict.keys()),
                                       binarizer=lb)
PERF_AXIS = pd.DataFrame(perf_list)

In [84]:
PERF_AXIS[['f1', 'precision', 'recall', 'specificity', 'model', 'Class']].groupby(['model', 'Class']).mean()

,,f1,precision,recall,specificity
model,Class,,,,
DT,Disease/Normal,0.840553,0.725219,0.999518,0.001064
LR,Disease/Normal,0.825318,0.746646,0.922887,0.173457
XGB,Disease/Normal,0.844815,0.796593,0.900007,0.391924


In [85]:
tree_utils.net_benefit_curve_plot(RES_AXIS_DF, true_col_prefix='Y_test',
                                     pred_col_prefix='Y_pred',
                                     output_path=os.path.join(data_path, fstring),
                                     threshold_steps=20, 
                                     xlim=[0,0.5],
                                     ylim=[-1,1],
                                     plot_title="Heart Axis")

tree_utils.calibration_curve_plot(RES_AXIS_DF,
                                   true_col_prefix='Y_test',
                                   pred_col_prefix='Y_pred',
                                   output_path=os.path.join(data_path, fstring),
                                   n_bins=10,
                                   plot_title="Heart Axis",
                                   show_plot=True)

T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:253: RuntimeWarning: invalid value encountered in scalar divide
  net_benefit = (tp - fp * threshold / (1 - threshold)) / N
T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:257: RuntimeWarning: divide by zero encountered in scalar divide
  all_positive_benefit = tp/N - threshold / (1 - threshold)*(1-prevalence)
T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:253: RuntimeWarning: invalid value encountered in scalar divide
  net_benefit = (tp - fp * threshold / (1 - threshold)) / N
T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:257: RuntimeWarning: divide by zero encountered in scalar divide
  all_positive_benefit = tp/N - threshold / (1 - threshold)*(1-prevalence)
T:\laupodteam\AIOS\Bram\notebooks\code_dev\m

# Muscle model

In [86]:
target_col = "Diagnosis"
target_inclusion = ['SR','LVH','Microvoltages']
Reduction_map = {'Microvoltages': 'Disease', 
                 'LVH': 'Disease',
                 'SR': 'Normal'}
if ALL_FEATURES:
    features_to_use = []
else:
    features_to_use = ['qrs_vector mean lead_0',
                     'qrs_ampl mean lead_0',
                     'qrs_vector mean lead_1',
                     'qrs_ampl mean lead_1',
                     'qrs_vector mean lead_2',
                     'qrs_ampl mean lead_2',
                     'qrs_vector mean lead_3',
                     'qrs_ampl mean lead_3',
                     'qrs_vector mean lead_4',
                     'qrs_ampl mean lead_4',
                     'qrs_vector mean lead_5',
                     'qrs_ampl mean lead_5',
                     'qrs_vector mean lead_6',
                     'qrs_ampl mean lead_6',
                     'qrs_vector mean lead_7',
                     'qrs_ampl mean lead_7',
                     'morphology lead_0',
                     'morphology lead_1',
                     'morphology lead_2',
                     'morphology lead_3',
                     'morphology lead_4',
                     'morphology lead_5',
                     'morphology lead_6',
                     'morphology lead_7']

In [87]:
_cat_enc = PipeOrdEncoder
for k in PipeDict.keys():
    PipeDict[k].set_params(CatEncoder= _cat_enc)

In [88]:
if len(features_to_use)==0:
    meas_cols = [c for c in DATA.columns if ('Dataset' not in c) 
                 & (target_col not in c)
                 & ("Heart Axis Diagnosis" not in c)]
else:
    meas_cols = features_to_use
    
fstring = f"MUSCLE_{CLASS_WEIGHT_STRING}{ALL_FEATURES_STRING}{REDUCED_LABEL_STRING}{CALIBRATED_CLASSIFIER_STRING}{FEATURE_COMBO_STRING}"
os.makedirs(os.path.join(data_path, fstring), exist_ok=True)

MUSCLE_DATA = DATA.loc[DATA[target_col].apply(lambda x: any([c in x for c in target_inclusion])), 
                       meas_cols+[target_col]+['Dataset']]

MUSCLE_DATA = MUSCLE_DATA.assign(Diagnosis=MUSCLE_DATA.Diagnosis.map({
                                                            'SR': 'SR',
                                                            'Microvoltages': 'Microvoltages',
                                                            'LVH': 'LVH',
                                                            'LAFB , LVH': 'LVH',
                                                            'Microvoltages , BF': 'Microvoltages',
                                                            'Microvoltages , RBBB': 'Microvoltages',
                                                            'Microvoltages , LAFB': 'Microvoltages',
                                                            'LVH , BF': 'LVH',
                                                            'LVH , RBBB': 'LVH',
                                                            'LVH , LBBB': 'LVH'
                                                        }))

if USE_REDUCED_LABELS:
    MUSCLE_DATA.loc[:, target_col] = MUSCLE_DATA[target_col].map(Reduction_map)
    
MUSCLE_DATA.to_parquet(os.path.join(data_path, fstring, f'DATA.parquet'))
MUSCLE_DATA = MUSCLE_DATA.drop('Dataset', axis=1)

## Training loop

In [89]:
splitter = RepeatedStratifiedKFold(n_splits=num_splits, n_repeats=num_repeats, random_state=7)
X = MUSCLE_DATA.iloc[:, :-1]
Y = MUSCLE_DATA.iloc[:,-1]

lb = LabelBinarizer()
lbe = LabelEncoder()
 #(RES_LIST_AXIS[0]['Y_test'])
Yenc = lbe.fit_transform(Y.values) #(RES_LIST_AXIS[0]['Y_test'])
lb.fit(Yenc)
ClassMap_MUSCLE = {i:c for i,c in enumerate(lbe.classes_)}

In [90]:
ClassMap_MUSCLE

{0: 'Disease', 1: 'Normal'}

In [91]:
RES_LIST_MUSCLE, RES_MUSCLE_DF, RES_MUSCLE_INDCS_DF =\
    tree_utils.training_loop(X, Yenc, splitter, PipeDict,  
                             use_class_weights=USE_CLASS_WEIGHT, 
                             ClassMap=ClassMap_MUSCLE, 
                             num_splits=num_splits, 
                             num_repeats=num_repeats, 
                             make_df=True)

100%|██████████| 100/100 [31:06<00:00, 18.67s/it]


In [92]:
RES_MUSCLE_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS.parquet"))
RES_MUSCLE_INDCS_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS_w_indices.parquet"))

### make roc and precision recall curves

In [93]:
Y.value_counts()

Diagnosis
Normal     649
Disease    413
Name: count, dtype: int64

In [94]:
n_classes = len(lb.classes_)
colors = ['blue', 'green', 'red', 'yellow', 'magenta', 'cyan', 'black']

In [95]:
PLOTS_MUSCLE = tree_utils.make_plots(RES_LIST_MUSCLE, lb,  n_classes, colors,
                                     ClassMap_MUSCLE,
                                     output_map=os.path.join(data_path, fstring),
                                     show_plot=False,
                                     models = list(PipeDict.keys()),
                                     plot_title="Heart Muscle")

perf_list = tree_utils.get_performance(RES_LIST_MUSCLE, 
                                       threshold=1/n_classes, 
                                       ClassMap=ClassMap_MUSCLE, 
                                       models = list(PipeDict.keys()),
                                       binarizer=lb)
PERF_MUSCLE = pd.DataFrame(perf_list)

In [96]:
PERF_MUSCLE[['f1', 'precision', 'recall', 'specificity', 'model', 'Class']].groupby(['model', 'Class']).mean()

,,f1,precision,recall,specificity
model,Class,,,,
DT,Disease/Normal,0.825686,0.779613,0.879337,0.606597
LR,Disease/Normal,0.799530,0.731810,0.882603,0.488926
XGB,Disease/Normal,0.852704,0.835365,0.872264,0.727172


In [97]:
tree_utils.net_benefit_curve_plot(RES_MUSCLE_DF, true_col_prefix='Y_test',
                                     pred_col_prefix='Y_pred',
                                     output_path=os.path.join(data_path, fstring),
                                     threshold_steps=20, 
                                     xlim=[0,0.5],
                                     ylim=[-1,1],
                                     plot_title="Heart Muscle")

tree_utils.calibration_curve_plot(RES_MUSCLE_DF,
                                   true_col_prefix='Y_test',
                                   pred_col_prefix='Y_pred',
                                   output_path=os.path.join(data_path, fstring),
                                   n_bins=10,
                                   plot_title="Heart Muscle",
                                   show_plot=True)

T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:253: RuntimeWarning: invalid value encountered in scalar divide
  net_benefit = (tp - fp * threshold / (1 - threshold)) / N
T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:257: RuntimeWarning: divide by zero encountered in scalar divide
  all_positive_benefit = tp/N - threshold / (1 - threshold)*(1-prevalence)
T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:253: RuntimeWarning: invalid value encountered in scalar divide
  net_benefit = (tp - fp * threshold / (1 - threshold)) / N
T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:257: RuntimeWarning: divide by zero encountered in scalar divide
  all_positive_benefit = tp/N - threshold / (1 - threshold)*(1-prevalence)
T:\laupodteam\AIOS\Bram\notebooks\code_dev\m

# Conduction model

In [98]:
target_col = "Diagnosis"
target_inclusion = ['BF', 'LBBB','RBBB','LAFB', 'SR']
Reduction_map = {'BF': 'Disease', 
                 'LBBB': 'Disease', 
                 'RBBB': 'Disease',
                 'LAFB': 'Disease',
                 'SR': 'Normal'}
features_to_use = []
_cat_enc = PipeOrdEncoder
for k in PipeDict.keys():
    PipeDict[k].set_params(CatEncoder= _cat_enc)

In [99]:
if len(features_to_use)==0:
    meas_cols = [c for c in DATA.columns if ('Dataset' not in c) 
                 & (target_col not in c)
                 & ("Heart Axis Diagnosis" not in c)]
else:
    meas_cols = features_to_use
    
fstring = f"CONDUCTION{CLASS_WEIGHT_STRING}{ALL_FEATURES_STRING}{REDUCED_LABEL_STRING}{CALIBRATED_CLASSIFIER_STRING}{FEATURE_COMBO_STRING}"
os.makedirs(os.path.join(data_path, fstring), exist_ok=True)

CONDUCTION_DATA = DATA.loc[DATA[target_col].apply(lambda x: any([c in x for c in target_inclusion])),  meas_cols+[target_col]+['Dataset']]

CONDUCTION_DATA = CONDUCTION_DATA.assign(Diagnosis=CONDUCTION_DATA.Diagnosis.map({
                                                                'SR': 'SR',
                                                                'BF': 'BF',
                                                                'RBBB': 'RBBB',
                                                                'LBBB': 'LBBB',
                                                                'LAFB': 'LAFB',
                                                                'LAFB , LVH': 'LAFB',
                                                                'Microvoltages , BF': 'BF',
                                                                'Microvoltages , RBBB': 'RBBB',
                                                                'Microvoltages , LAFB': 'LAFB', 
                                                                'LVH , BF': 'BF',
                                                                'LVH , RBBB': 'RBBB',
                                                                'LVH , LBBB': 'LBBB'
                                                            }))
if USE_REDUCED_LABELS:
    CONDUCTION_DATA.loc[:, target_col] = CONDUCTION_DATA[target_col].map(Reduction_map)
    
CONDUCTION_DATA.to_parquet(os.path.join(data_path, fstring, 'CONDUCTION.parquet'))
CONDUCTION_DATA = CONDUCTION_DATA.drop('Dataset', axis=1)

## Training loop

In [100]:
splitter = RepeatedStratifiedKFold(n_splits=num_splits, n_repeats=num_repeats, random_state=7)
X = CONDUCTION_DATA.iloc[:, :-1]
Y = CONDUCTION_DATA.iloc[:,-1]

lb = LabelBinarizer()
lbe = LabelEncoder()
Yenc = lbe.fit_transform(Y)
lb.fit(Yenc)    
ClassMap_CONDUCTION = {i:c for i,c in enumerate(lbe.classes_)}

In [101]:
ClassMap_CONDUCTION

{0: 'Disease', 1: 'Normal'}

In [102]:
RES_LIST_CONDUCTION, RES_CONDUCTION_DF, RES_CONDUCTION_INDCS_DF = tree_utils.training_loop(X, Yenc, splitter, PipeDict,
                                    use_class_weights=USE_CLASS_WEIGHT, 
                                    ClassMap=ClassMap_CONDUCTION,
                                    num_splits=num_splits,num_repeats=num_repeats,
                                    make_df=True)

100%|██████████| 100/100 [1:26:52<00:00, 52.12s/it]


In [103]:
RES_CONDUCTION_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS.parquet"))
RES_CONDUCTION_INDCS_DF.to_parquet(os.path.join(data_path, fstring, "RESULTS_w_indices.parquet"))

### make roc and precision recall curves

In [104]:
Y.value_counts()

Diagnosis
Disease    702
Normal     649
Name: count, dtype: int64

In [105]:
n_classes = len(lb.classes_)
colors = ['blue', 'green', 'red', 'yellow', 'magenta', 'cyan', 'black']

In [106]:
PLOTS_CONDUCTION = tree_utils.make_plots(RES_LIST_CONDUCTION, lb,  n_classes, colors,
                                         ClassMap_CONDUCTION,
                                         output_map=os.path.join(data_path, fstring),
                                         show_plot=False,
                                         models = list(PipeDict.keys()),
                                         plot_title="Heart Conduction")

perf_list = tree_utils.get_performance(RES_LIST_CONDUCTION,
                                       threshold=1/n_classes, 
                                       ClassMap=ClassMap_CONDUCTION,
                                       models = list(PipeDict.keys()),
                                       binarizer=lb)

PERF_CONDUCTION = pd.DataFrame(perf_list)

In [107]:
PERF_CONDUCTION[['f1', 'precision', 'recall', 'specificity', 'model', 'Class']].groupby(['model', 'Class']).mean()

,,f1,precision,recall,specificity
model,Class,,,,
DT,Disease/Normal,0.889252,0.879403,0.900767,0.884330
LR,Disease/Normal,0.868216,0.826392,0.916163,0.820252
XGB,Disease/Normal,0.907831,0.900950,0.916031,0.905861


In [108]:
tree_utils.net_benefit_curve_plot(RES_CONDUCTION_DF, true_col_prefix='Y_test',
                                     pred_col_prefix='Y_pred',
                                     output_path=os.path.join(data_path, fstring),
                                     threshold_steps=20, 
                                     xlim=[0,0.5],
                                     ylim=[-1,1],
                                     plot_title="Heart Conduction")

tree_utils.calibration_curve_plot(RES_CONDUCTION_DF,
                                   true_col_prefix='Y_test',
                                   pred_col_prefix='Y_pred',
                                   output_path=os.path.join(data_path, fstring),
                                   n_bins=10,
                                   plot_title="Heart Conduction",
                                   show_plot=True)

T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:253: RuntimeWarning: invalid value encountered in scalar divide
  net_benefit = (tp - fp * threshold / (1 - threshold)) / N
T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:257: RuntimeWarning: divide by zero encountered in scalar divide
  all_positive_benefit = tp/N - threshold / (1 - threshold)*(1-prevalence)
T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:253: RuntimeWarning: invalid value encountered in scalar divide
  net_benefit = (tp - fp * threshold / (1 - threshold)) / N
T:\laupodteam\AIOS\Bram\notebooks\code_dev\miniECG_interpretation\TreeBuilder\sandbox\..\src\tree_utils.py:257: RuntimeWarning: divide by zero encountered in scalar divide
  all_positive_benefit = tp/N - threshold / (1 - threshold)*(1-prevalence)
T:\laupodteam\AIOS\Bram\notebooks\code_dev\m